<a href="https://colab.research.google.com/github/RuthKassahun/Brain_Tissue_Segmentation/blob/main/2D_unet_4versions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import glob
import os

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import cv2
import glob
import warnings
import scipy.misc
!pip install simpleitk
import SimpleITK as sitk
from scipy import ndimage
import matplotlib.pyplot as plt
from keras.models import Model

from keras.preprocessing.image import ImageDataGenerator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [113]:
import cv2
import glob
import warnings
import scipy.misc
import numpy as np
import nibabel as nib
!pip install simpleitk
import SimpleITK as sitk
from scipy import ndimage
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, Input

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [112]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.8 MB/s eta 0:00:00


In [7]:
!pip install keras_unet_collection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 KB 3.3 MB/s eta 0:00:00


In [41]:
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, Input

In [93]:
def get_unet(img_size=PATCH_SIZE, n_classes=N_CLASSES, n_input_channels=N_INPUT_CHANNELS, scale=1):
    inputs = keras.Input(shape=img_size + (n_input_channels, ))

    # Encoding path
    conv1 = layers.Conv2D(32*scale, (3, 3), padding="same", activation='relu')(inputs)
    max1 = layers.MaxPooling2D((2, 2))(conv1)

    conv2 = layers.Conv2D(64*scale, (3, 3), padding="same", activation='relu')(max1)
    max2 = layers.MaxPooling2D((2, 2))(conv2)

    conv3 = layers.Conv2D(128*scale, (3, 3), padding="same", activation='relu')(max2)
    max3 = layers.MaxPooling2D((2, 2))(conv3)

    lat = layers.Conv2D(256*scale, (3, 3), padding="same", activation='relu')(max3)

    # Decoding path
    up1 = layers.UpSampling2D((2, 2))(lat)
    concat1 = layers.concatenate([conv3, up1], axis=-1)
    conv4 = layers.Conv2D(128*scale, (3, 3), padding="same", activation='relu')(concat1)
    
    up2 = layers.UpSampling2D((2, 2))(conv4)
    concat2 = layers.concatenate([conv2, up2], axis=-1)
    conv5 = layers.Conv2D(64*scale, (3, 3), padding="same", activation='relu')(concat2)
    
    up3 = layers.UpSampling2D((2, 2))(conv5)
    concat3 = layers.concatenate([conv1, up3], axis=-1)

    conv6 = layers.Conv2D(32*scale, (3, 3), padding="same", activation='relu')(concat3)

    outputs = layers.Conv2D(n_classes, (1, 1), activation="softmax")(conv6)

    model = keras.Model(inputs, outputs)

    return model

In [165]:
def get_unet2(img_size=PATCH_SIZE, n_classes=N_CLASSES, n_input_channels=N_INPUT_CHANNELS, scale=1):
    inputs = keras.Input(shape=img_size + (n_input_channels, ))

    # Encoding path
    conv1 = layers.Conv2D(32*scale, (3, 3), padding="same", activation='relu')(inputs)
    max1 = layers.MaxPooling2D((2, 2))(conv1)

    conv2 = layers.Conv2D(64*scale, (3, 3), padding="same", activation='relu')(max1)
    max2 = layers.MaxPooling2D((2, 2))(conv2)

    conv3 = layers.Conv2D(128*scale, (3, 3), padding="same", activation='relu')(max2)
    max3 = layers.MaxPooling2D((2, 2))(conv3)

    conv4 = layers.Conv2D(256*scale, (3, 3), padding="same", activation='relu')(max3)
    max4 = layers.MaxPooling2D((2, 2))(conv4)

    lat = layers.Conv2D(512*scale, (3, 3), padding="same", activation='relu')(max4)

    # Decoding path
    up1 = layers.UpSampling2D((2, 2))(lat)
    concat1 = layers.concatenate([conv4, up1], axis=-1)
    conv5 = layers.Conv2D(256*scale, (3, 3), padding="same", activation='relu')(concat1)
    
    up2 = layers.UpSampling2D((2, 2))(conv5)
    concat2 = layers.concatenate([conv3, up2], axis=-1)
    conv6 = layers.Conv2D(128*scale, (3, 3), padding="same", activation='relu')(concat2)
    
    up3 = layers.UpSampling2D((2, 2))(conv6)
    concat3 = layers.concatenate([conv2, up3], axis=-1)
    conv7 = layers.Conv2D(64*scale, (3, 3), padding="same", activation='relu')(concat3)

    up4 = layers.UpSampling2D((2, 2))(conv7)
    concat4 = layers.concatenate([conv1, up4], axis=-1)
    conv8 = layers.Conv2D(32*scale, (3, 3), padding="same", activation='relu')(concat4)

    outputs = layers.Conv2D(n_classes, (1, 1), activation="softmax")(conv8)

    model = keras.Model(inputs, outputs)

    return model

In [237]:
def get_unet3(img_size=PATCH_SIZE, n_classes=N_CLASSES, n_input_channels=N_INPUT_CHANNELS, scale=1):
    inputs = keras.Input(shape=img_size + (n_input_channels, ))

    # Encoding path
    conv1 = layers.Conv2D(32*scale, (3, 3), padding="same", activation='relu')(inputs)
    drop1 = layers.Dropout(rate=dropout_rate)(conv1, training=True)
    max1 = layers.MaxPooling2D((2, 2))(drop1)

    conv2 = layers.Conv2D(64*scale, (3, 3), padding="same", activation='relu')(max1)
    drop2 = layers.Dropout(rate=dropout_rate)(conv2, training=True)
    max2 = layers.MaxPooling2D((2, 2))(drop2)

    conv3 = layers.Conv2D(128*scale, (3, 3), padding="same", activation='relu')(max2)
    drop3 = layers.Dropout(rate=dropout_rate)(conv3, training=True)
    max3 = layers.MaxPooling2D((2, 2))(drop3)

    conv4 = layers.Conv2D(256*scale, (3, 3), padding="same", activation='relu')(max3)
    drop4 = layers.Dropout(rate=dropout_rate)(conv4, training=True)
    max4 = layers.MaxPooling2D((2, 2))(drop4)

    lat = layers.Conv2D(512*scale, (3, 3), padding="same", activation='relu')(max4)
    drop5 = layers.Dropout(rate=dropout_rate)(lat, training=True)

    # Decoding path
    up1 = layers.UpSampling2D((2, 2))(drop5)
    concat1 = layers.concatenate([conv4, up1], axis=-1)
    conv5 = layers.Conv2D(256*scale, (3, 3), padding="same", activation='relu')(concat1)
    drop6 = layers.Dropout(rate=dropout_rate)(conv5, training=True)
    
    up2 = layers.UpSampling2D((2, 2))(drop6)
    concat2 = layers.concatenate([conv3, up2], axis=-1)
    conv6 = layers.Conv2D(128*scale, (3, 3), padding="same", activation='relu')(concat2)
    drop7 = layers.Dropout(rate=dropout_rate)(conv6, training=True)
    
    up3 = layers.UpSampling2D((2, 2))(drop7)
    concat3 = layers.concatenate([conv2, up3], axis=-1)
    conv7 = layers.Conv2D(64*scale, (3, 3), padding="same", activation='relu')(concat3)
    drop8 = layers.Dropout(rate=dropout_rate)(conv7, training=True)

    up4 = layers.UpSampling2D((2, 2))(drop8)
    concat4 = layers.concatenate([conv1, up4], axis=-1)
    conv8 = layers.Conv2D(32*scale, (3, 3), padding="same", activation='relu')(concat4)
    drop9 = layers.Dropout(rate=dropout_rate)(conv8, training=True)

    outputs = layers.Conv2D(n_classes, (1, 1), activation="softmax")(conv8)

    model = keras.Model(inputs, outputs)

    return model

In [248]:
def get_unet4(img_size=PATCH_SIZE, n_classes=N_CLASSES, n_input_channels=N_INPUT_CHANNELS, scale=1):
    inputs = keras.Input(shape=img_size + (n_input_channels, ))

    # Encoding path
    conv1 = layers.BatchNormalization()(layers.Conv2D(32*scale, (3, 3), padding="same", activation='relu')(inputs))
    drop1 = layers.Dropout(rate=dropout_rate)(conv1, training=True)
    max1 = layers.MaxPooling2D((2, 2))(drop1)

    conv2 = layers.BatchNormalization()(layers.Conv2D(64*scale, (3, 3), padding="same", activation='relu')(max1))
    drop2 = layers.Dropout(rate=dropout_rate)(conv2, training=True)
    max2 = layers.MaxPooling2D((2, 2))(drop2)

    conv3 = layers.BatchNormalization()(layers.Conv2D(128*scale, (3, 3), padding="same", activation='relu')(max2))
    drop3 = layers.Dropout(rate=dropout_rate)(conv3, training=True)
    max3 = layers.MaxPooling2D((2, 2))(drop3)

    conv4 = layers.BatchNormalization()(layers.Conv2D(256*scale, (3, 3), padding="same", activation='relu')(max3))
    drop4 = layers.Dropout(rate=dropout_rate)(conv4, training=True)
    max4 = layers.MaxPooling2D((2, 2))(drop4)

    lat = layers.BatchNormalization()(layers.Conv2D(512*scale, (3, 3), padding="same", activation='relu')(max4))
    drop5 = layers.Dropout(rate=dropout_rate)(lat, training=True)

    # Decoding path
    up1 = layers.UpSampling2D((2, 2))(drop5)
    concat1 = layers.concatenate([conv4, up1], axis=-1)
    conv5 = layers.BatchNormalization()(layers.Conv2D(256*scale, (3, 3), padding="same", activation='relu')(concat1))
    drop6 = layers.Dropout(rate=dropout_rate)(conv5, training=True)
    
    up2 = layers.UpSampling2D((2, 2))(drop6)
    concat2 = layers.concatenate([conv3, up2], axis=-1)
    conv6 = layers.BatchNormalization()(layers.Conv2D(128*scale, (3, 3), padding="same", activation='relu')(concat2))
    drop7 = layers.Dropout(rate=dropout_rate)(conv6, training=True)
    
    up3 = layers.UpSampling2D((2, 2))(drop7)
    concat3 = layers.concatenate([conv2, up3], axis=-1)
    conv7 = layers.BatchNormalization()(layers.Conv2D(64*scale, (3, 3), padding="same", activation='relu')(concat3))
    drop8 = layers.Dropout(rate=dropout_rate)(conv7, training=True)

    up4 = layers.UpSampling2D((2, 2))(drop8)
    concat4 = layers.concatenate([conv1, up4], axis=-1)
    conv8 = layers.BatchNormalization()(layers.Conv2D(32*scale, (3, 3), padding="same", activation='relu')(concat4))
    drop9 = layers.Dropout(rate=dropout_rate)(conv8, training=True)

    outputs = layers.Conv2D(n_classes, (1, 1), activation="softmax")(conv8)

    model = keras.Model(inputs, outputs)

    return model

In [138]:
!pip install focal_loss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [139]:
from focal_loss import BinaryFocalLoss

In [145]:
from focal_loss import sparse_categorical_focal_loss

In [235]:
FNAME_PATTERN = '/content/drive/MyDrive/MISA_Final_project/IBSR-{}-{}.nii'
N_VOLUMES = 15
IMAGE_SIZE = (256, 128, 256)

# network parameters
N_CLASSES = 4
N_INPUT_CHANNELS = 1
PATCH_SIZE = (32, 32)
PATCH_STRIDE = (32, 32)

# training, validation, test parameters
TRAINING_VOLUMES = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
VALIDATION_VOLUMES = [10, 11, 12, 13, 14]
#TEST_VOLUMES = [14]


# data preparation parameters
CONTENT_THRESHOLD = 0.3

# training parameters
N_EPOCHS = 500
BATCH_SIZE = 32
PATIENCE = 20
MODEL_FNAME_PATTERN = 'model_v3.h5'
# OPTIMISER = 'Adam'
learning_rate=0.0001
OPTIMISER = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)
# OPTIMISER = tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate)
LOSS = 'categorical_crossentropy'
# LOSS ='sparse_categorical_crossentropy'
# LOSS = tfa.losses.GIoULoss()
#LOSS = BinaryFocalLoss(gamma=2)
# LOSS = sparse_categorical_focal_loss(from_logits=False, class_weight=[0.1, 0.9, 1, 1])

# LOSS = tf.keras.losses.SparseCategoricalCrossentropy()

dropout_rate = 0.5

In [226]:
def load_data(n_volumes=N_VOLUMES, image_size=IMAGE_SIZE, fname_pattern=FNAME_PATTERN) :
  T1_volumes = np.zeros((n_volumes, *image_size, 1))
  #T2_volumes = np.zeros((n_volumes, *image_size, 1))
  labels = np.zeros((n_volumes, *image_size, 1))
  for i in range(n_volumes) :
    img_data = nib.load(fname_pattern.format(i+1, 'T1'))
    T1_volumes[i] = img_data.get_fdata()

    # img_data = nib.load(fname_pattern.format(i+1, 'T2'))
    # T2_volumes[i] = img_data.get_fdata()

    seg_data = nib.load(fname_pattern.format(i+1, 'label'))
    labels[i] = seg_data.get_fdata()

  return (T1_volumes, labels)

In [227]:
(T1_volumes, labels) = load_data()

In [228]:
training_volumes_T1 = T1_volumes[TRAINING_VOLUMES]
#training_volumes_T2 = T2_volumes[TRAINING_VOLUMES]
training_labels = labels[TRAINING_VOLUMES]

validation_volumes_T1 = T1_volumes[VALIDATION_VOLUMES]
#validation_volumes_T2 = T2_volumes[VALIDATION_VOLUMES]
validation_labels = labels[VALIDATION_VOLUMES]

#testing_volumes_T1 = T1_volumes[TEST_VOLUMES]
#testing_volumes_T2 = T2_volumes[TEST_VOLUMES]
#testing_labels = labels[TEST_VOLUMES]

In [229]:
def extract_patches(x, patch_size, patch_stride) :
  return tf.image.extract_patches(
    x,
    sizes=[1, *patch_size, 1],
    strides=[1, *patch_stride, 1],
    rates=[1, 1, 1, 1],
    padding='SAME', name=None)

In [230]:
def extract_useful_patches(
    volumes, labels,
    image_size=IMAGE_SIZE,
    patch_size=PATCH_SIZE,
    stride=PATCH_STRIDE,
    threshold=CONTENT_THRESHOLD,
    num_classes=N_CLASSES) :
  volumes = volumes.reshape([-1, image_size[1], image_size[2], 1])
  labels = labels.reshape([-1, image_size[1], image_size[2], 1])


  vol_patches = extract_patches(volumes, patch_size, stride).numpy()
  seg_patches = extract_patches(labels, patch_size, stride).numpy()
  
  vol_patches = vol_patches.reshape([-1, *patch_size, 1])
  seg_patches = seg_patches.reshape([-1, *patch_size, ])

  foreground_mask = seg_patches != 0 

  useful_patches = foreground_mask.sum(axis=(1, 2)) > threshold * np.prod(patch_size)

  vol_patches = vol_patches[useful_patches]
  seg_patches = seg_patches[useful_patches]
  print(np.unique(seg_patches))
  seg_patches = tf.keras.utils.to_categorical(seg_patches, num_classes=N_CLASSES, dtype='float32')
  
  print(np.unique(seg_patches))
  return (vol_patches, seg_patches)

In [231]:
# extract patches from training set
(training_patches_T1, training_patches_seg) = extract_useful_patches(training_volumes_T1, training_labels)
#(training_patches_T2, _) = extract_useful_patches(training_volumes_T2, training_labels)

# extract patches from validation set
(validation_patches_T1, validation_patches_seg) = extract_useful_patches(validation_volumes_T1, validation_labels)
#(validation_patches_T2, _) = extract_useful_patches(validation_volumes_T2, validation_labels)

[0. 1. 2. 3.]
[0. 1.]
[0. 1. 2. 3.]
[0. 1.]


In [232]:
X_train=np.dstack([training_patches_T1] * 3)
X_train = X_train.reshape(-1, 32,32,3)
# X_train = np.rollaxis(X_train, 3, 1)

X_val=np.dstack([validation_patches_T1] * 3)
X_val= X_val.reshape(-1, 32,32,3)
# X_val = np.rollaxis(X_val, 3, 1)
print(validation_patches_T1.shape)
print(X_val.shape)

print(training_patches_T1.shape)
print(X_train.shape)

(6648, 32, 32, 1)
(6648, 32, 32, 3)
(12747, 32, 32, 1)
(12747, 32, 32, 3)


In [249]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=PATIENCE), # early stopping
    tf.keras.callbacks.ModelCheckpoint(filepath=MODEL_FNAME_PATTERN, save_best_only=True) # save the best based on validation
]

unet = get_unet4()
unet.compile(optimizer=OPTIMISER, loss=LOSS, 
             metrics=tf.keras.metrics.MeanIoU(num_classes=4)
            )
unet.fit(
    x=training_patches_T1,
    # x=X_train, 
    y=training_patches_seg,
    # validation_data=(X_val, validation_patches_seg),
    validation_data=(validation_patches_T1, validation_patches_seg),
    batch_size=BATCH_SIZE,
    epochs=500,
    callbacks=my_callbacks,
    verbose=1)

Epoch 1/500
399/399 [==============================] - 7s 14ms/step - loss: 0.4352 - mean_io_u_21: 0.3750 - val_loss: 0.6703 - val_mean_io_u_21: 0.3779
Epoch 2/500
399/399 [==============================] - 5s 12ms/step - loss: 0.3055 - mean_io_u_21: 0.3750 - val_loss: 0.5894 - val_mean_io_u_21: 0.3775
Epoch 3/500
399/399 [==============================] - 5s 12ms/step - loss: 0.2771 - mean_io_u_21: 0.3750 - val_loss: 0.5337 - val_mean_io_u_21: 0.3777
Epoch 4/500
399/399 [==============================] - 5s 13ms/step - loss: 0.2543 - mean_io_u_21: 0.3750 - val_loss: 0.5079 - val_mean_io_u_21: 0.3786
Epoch 5/500
399/399 [==============================] - 5s 12ms/step - loss: 0.2396 - mean_io_u_21: 0.3750 - val_loss: 0.6551 - val_mean_io_u_21: 0.3807
Epoch 6/500
399/399 [==============================] - 5s 12ms/step - loss: 0.2282 - mean_io_u_21: 0.3750 - val_loss: 0.6677 - val_mean_io_u_21: 0.3818
Epoch 7/500
399/399 [==============================] - 5s 12ms/step - loss: 0.2187 - mea

In [239]:
# validation_volumes_T1_processed = validation_volumes_T1.reshape([-1, IMAGE_SIZE[1], IMAGE_SIZE[2], 1])
# validation_labels_processed = validation_labels.reshape([-1, IMAGE_SIZE[1], IMAGE_SIZE[2]])

In [240]:
validation_volumes_T1_processed = validation_volumes_T1.reshape([-1, PATCH_SIZE[0], PATCH_SIZE[1], 1])
validation_labels_processed = validation_labels.reshape([-1, PATCH_SIZE[0], PATCH_SIZE[1]])
validation_labels_processed=tf.keras.utils.to_categorical(validation_labels_processed, num_classes=4, dtype='float32')

In [241]:
testing_volumes_T1 = validation_volumes_T1[1]
testing_labels = validation_labels[1]
testing_volumes_T1_processed = testing_volumes_T1.reshape([-1, PATCH_SIZE[0], PATCH_SIZE[1], 1])
testing_labels_processed = testing_labels.reshape([-1, PATCH_SIZE[0], PATCH_SIZE[1]])
# testing_labels_processed=tf.keras.utils.to_categorical(testing_labels_processed, num_classes=4, dtype='float32')

In [242]:
testing_volumes_T1 = validation_volumes_T1[1]
testing_labels = validation_labels[1]
testing_volumes_T1_processed = testing_volumes_T1.reshape([-1, IMAGE_SIZE[1], IMAGE_SIZE[2], 1])
testing_labels_processed = testing_labels.reshape([-1, IMAGE_SIZE[1], IMAGE_SIZE[2]])

In [251]:
unet = get_unet4(
    img_size=(IMAGE_SIZE[1], IMAGE_SIZE[2]),
    n_classes=N_CLASSES,
    n_input_channels=N_INPUT_CHANNELS)
unet.compile(optimizer=OPTIMISER, loss=LOSS)
unet.load_weights(MODEL_FNAME_PATTERN)

In [252]:
def pred_val_data(x)  :
  # creates probability map of each label for all volumes
  #prediction = unet.predict(x=testing_volumes_processed)
  prediction = unet.predict(x)
  # print(np.unique(prediction))
  # print(prediction.shape)

  prediction = np.argmax(prediction, axis=3)

  # plt.axis('off')
  # plt.imshow(prediction[:, :, 150])

  return prediction

In [253]:
prediction = pred_val_data(testing_volumes_T1_processed)

8/8 [==============================] - 1s 15ms/step


In [28]:
!pip install medpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.8/151.8 KB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for medpy: filename=MedPy-0.4.0-cp38-cp38-linux_x86_64.whl size=753423 sha256=b345fbb64c15f7ed64336bca696ae5b233e5405f1c5e19ed79dcd830e77bd3ff
  Stored in directory: /root/.cache/pip/wheels/fc/38/7d/e0b8bcb2770f779a93cab5ab7fa6dd344011e1278cb90cab86
Successfully built medpy


In [254]:
import numpy as np
import nibabel as nib
from medpy.metric.binary import dc, hd, ravd

def compute_dice(prediction, reference) :
  for c in np.unique(prediction) :
    dsc_val = dc(prediction == c, reference==c)
    print(f'Dice coefficient class {c} equal to {dsc_val : .2f}')

def compute_hd(prediction, reference, voxel_spacing) :
  for c in np.unique(prediction) :
    hd_val = hd(prediction == c, reference==c, voxelspacing=voxel_spacing, connectivity=1)
    print(f'Hausdorff distance class {c} equal to {hd_val : .2f}')

def compute_ravd(prediction, reference) :
  for c in np.unique(prediction) :
    ravd_val = ravd(prediction == c, reference==c)
    print(f'Dice coefficient class {c} avd {ravd_val : .2f}')

compute_dice(prediction, testing_labels_processed)
compute_hd(prediction, testing_labels_processed, [1, 1, 1])
compute_ravd(prediction, testing_labels_processed)

Dice coefficient class 0 equal to  0.99
Dice coefficient class 1 equal to  0.76
Dice coefficient class 2 equal to  0.88
Dice coefficient class 3 equal to  0.74
Hausdorff distance class 0 equal to  43.00
Hausdorff distance class 1 equal to  87.17
Hausdorff distance class 2 equal to  97.10


KeyboardInterrupt: ignored

In [247]:
import numpy as np
import nibabel as nib
from medpy.metric.binary import dc, hd, ravd

def compute_dice(prediction, reference) :
  dice_scores = np.zeros((1,4))
  for c in np.unique(prediction) :
    dsc_val = dc(prediction == c, reference==c)
    dice_scores[0,c] = dsc_val
    print(f'Dice coefficient class {c} equal to {dsc_val : .2f}')
  return dice_scores

def compute_hd(prediction, reference, voxel_spacing) :
  hd_scores = np.zeros((1,4))
  for c in np.unique(prediction) :
    hd_val = hd(prediction == c, reference==c, voxelspacing=voxel_spacing, connectivity=1)
    hd_scores[0,c] = hd_val
    print(f'Hausdorff distance class {c} equal to {hd_val : .2f}')
  return hd_scores

def compute_ravd(prediction, reference) :
  ravd_scores = np.zeros((1,4))
  for c in np.unique(prediction) :
    ravd_val = ravd(prediction == c, reference==c)
    ravd_scores[0,c] = ravd_val
    print(f'Dice coefficient class {c} avd {ravd_val : .2f}')
  return ravd_scores

# compute_dice(prediction, testing_labels_processed)
# compute_hd(prediction, testing_labels_processed, [1, 1, 1])
# compute_ravd(prediction, testing_labels_processed)

dice_list=np.zeros((len(validation_volumes_T1),4))
hd_list=np.zeros((len(validation_volumes_T1),4))
ravd_list=np.zeros((len(validation_volumes_T1),4))

for i in range(0,len(validation_volumes_T1)):

  testing_volumes_T1=validation_volumes_T1[i]
  testing_labels=validation_labels[i]
  testing_volumes_T1_processed = testing_volumes_T1.reshape([-1, IMAGE_SIZE[1], IMAGE_SIZE[2], 1])
  testing_labels_processed = testing_labels.reshape([-1, IMAGE_SIZE[1], IMAGE_SIZE[2]])
  prediction = pred_val_data(testing_volumes_T1_processed)

  dice_list[i,:] = compute_dice(prediction, testing_labels_processed)
  hd_list[i,:] = compute_hd(prediction, testing_labels_processed, [1, 1, 1])
  ravd_list[i,:] = compute_ravd(prediction, testing_labels_processed)

dice_total=dice_list.sum(axis=0)
hd_total=hd_list.sum(axis=0)
ravd_total=ravd_list.sum(axis=0)

for i in range(0,4):
  print("Average Dice, HD, and RAVD for class {} is {},{}, and {}".format(i,dice_total[i]/len(validation_volumes_T1),hd_total[i]/len(validation_volumes_T1),ravd_total[i]/len(validation_volumes_T1)))

8/8 [==============================] - 0s 17ms/step
Dice coefficient class 0 equal to  1.00
Dice coefficient class 1 equal to  0.72
Dice coefficient class 2 equal to  0.89
Dice coefficient class 3 equal to  0.90
Hausdorff distance class 0 equal to  43.20
Hausdorff distance class 1 equal to  50.25
Hausdorff distance class 2 equal to  47.46
Hausdorff distance class 3 equal to  12.69
Dice coefficient class 0 avd -0.00
Dice coefficient class 1 avd  0.11
Dice coefficient class 2 avd -0.07
Dice coefficient class 3 avd  0.11
8/8 [==============================] - 0s 17ms/step
Dice coefficient class 0 equal to  1.00
Dice coefficient class 1 equal to  0.71
Dice coefficient class 2 equal to  0.90
Dice coefficient class 3 equal to  0.87
Hausdorff distance class 0 equal to  18.38
Hausdorff distance class 1 equal to  23.60
Hausdorff distance class 2 equal to  14.46
Hausdorff distance class 3 equal to  7.81
Dice coefficient class 0 avd  0.00
Dice coefficient class 1 avd -0.25
Dice coefficient class 

In [ ]:
dice_list

In [ ]:
hd_list


In [ ]:
ravd_list

In [ ]:
for i in range(0,4):
  print("Average Dice, HD, and RAVD for class {} is {},{}, and {}".format(i,dice_total[i]/len(validation_volumes_T1),hd_total[i]/len(validation_volumes_T1),ravd_total[i]/len(validation_volumes_T1)))